## 1. Framing the Problem 

In [1]:
from __future__ import print_function               
import keras 
from keras.models import Sequential                 
from keras.layers import Dense, Activation          
                                                    
from keras.optimizers import SGD                    

import pandas   
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

c:\users\asus\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 2. Get the data

In [2]:
dataset = pandas.read_csv("train.csv") 
dataset = dataset.as_matrix() 
X,y = dataset[:,1:], dataset[:,0] 

## 3. Explore the data

## 4. Prepare the data

In [3]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.20)
x_train = x_train/255.0                         
x_val = x_val/255.0
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_val = keras.utils.to_categorical(y_val, num_classes=10)

## 5.1 Create the model

In [4]:
# set parameters
batch_size = 256
epochs = 20
learning_rate = 0.001

# build the model
model = Sequential()                                         
model.add(Dense(300, activation='relu',input_dim=784))       
model.add(Dense(300, activation='relu'))                     
model.add(Dense(300, activation='relu')) 
model.add(Dense(10, activation='softmax'))                   
model.summary()                                              

##Using SGD as optimizer
my_optimizer = keras.optimizers.RMSprop(lr=learning_rate)                   
model.compile(optimizer=my_optimizer,                       
              loss='categorical_crossentropy',              
              metrics=['accuracy'])                         

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 300)               235500    
_________________________________________________________________
dense_2 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_3 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_4 (Dense)              (None, 10)                3010      
Total params: 419,110
Trainable params: 419,110
Non-trainable params: 0
_________________________________________________________________


## 5.2 train the model

In [5]:
best_weights_filepath = './best_weights2.hdf5' 
                                            
earlyStopping = keras.callbacks.EarlyStopping(monitor='val_acc',
                                              patience = 5) 

SaveBestWeights = keras.callbacks.ModelCheckpoint(best_weights_filepath,
                                                  monitor='val_acc',
                                                  save_best_only=True)


model_history = model.fit(x_train, y_train,                  
                    batch_size=batch_size,                   
                    epochs=epochs,                           
                    verbose= 2,                             
                    validation_data = (x_val, y_val),   
                    callbacks = [earlyStopping, SaveBestWeights]
                         )     
model.load_weights(best_weights_filepath) 

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
 - 3s - loss: 0.4198 - acc: 0.8721 - val_loss: 0.2907 - val_acc: 0.9073
Epoch 2/20
 - 3s - loss: 0.1495 - acc: 0.9534 - val_loss: 0.2303 - val_acc: 0.9318
Epoch 3/20
 - 3s - loss: 0.0913 - acc: 0.9713 - val_loss: 0.1897 - val_acc: 0.9393
Epoch 4/20
 - 2s - loss: 0.0622 - acc: 0.9806 - val_loss: 0.1730 - val_acc: 0.9529
Epoch 5/20
 - 2s - loss: 0.0454 - acc: 0.9855 - val_loss: 0.1251 - val_acc: 0.9660
Epoch 6/20
 - 2s - loss: 0.0338 - acc: 0.9896 - val_loss: 0.1550 - val_acc: 0.9604
Epoch 7/20
 - 2s - loss: 0.0273 - acc: 0.9916 - val_loss: 0.1114 - val_acc: 0.9740
Epoch 8/20
 - 4s - loss: 0.0192 - acc: 0.9939 - val_loss: 0.2547 - val_acc: 0.9364
Epoch 9/20
 - 3s - loss: 0.0166 - acc: 0.9949 - val_loss: 0.2775 - val_acc: 0.9454
Epoch 10/20
 - 3s - loss: 0.0150 - acc: 0.9952 - val_loss: 0.1555 - val_acc: 0.9695
Epoch 11/20
 - 3s - loss: 0.0104 - acc: 0.9967 - val_loss: 0.1390 - val_acc: 0.9732
Epoch 12/20
 - 4s - loss: 0.0119 - a

## 6. Evaluate the model:Submission to Kaggle

In [6]:
testset = pandas.read_csv("test.csv")             
testset = testset.as_matrix()                     
testset = testset/255.0                             
predictions = model.predict_classes(testset)           
submission2 = pandas.DataFrame(data=predictions, index=np.arange(1,len(predictions)+1), columns=['Label']) 
submission2.index.name = 'ImageId' 

csv_text = submission2.to_csv() 


with open("submission2.csv",'w') as csv_file:
    csv_file.write(csv_text)